In [1]:
import scipy.io as sio  
import matplotlib.pyplot as plt  
import numpy as np 
from numpy.linalg import *
import pandas as pd

In [2]:
def LDA_tr( x , lamb=0.1 ):
    num = np.zeros(len(x))
    m = np.zeros( ( len(x) , np.shape(x[0])[1] ) )
    S = np.zeros( (np.shape(x[0])[1]  , np.shape(x[0])[1]  , len(x)) )
    for i in range(len(x)):
        m[i,:] = x[i].mean(0).T
        num[i] = np.shape(x[i])[0]
        k = x[i] - m[i,:]
        S[:,:,i] = np.dot( k.T,k )
    
    Sw = S.mean(2)
    M = np.dot( num , m )/num.sum()
    Sb = np.dot(num*(m-M).T , (m-M))
    w = np.dot( inv( Sw + lamb*np.eye(np.shape(Sw)[1]) ),Sb )
    [a,b] = eig(w)
    w = b[:,:6].real
    flag = np.dot(m , w)
    return w,flag


In [3]:
def LDA_te( x , y , w , flag ):
    t = np.dot(test_x , w)
    dis = np.zeros((np.shape(test_x)[0],len(train_x)))
    for i in range(len(train_x)):
        k = t - flag[i,:]
        dis[:,i] = np.diag(np.dot(k,k.T))
    t = dis.argmin(1)+1
    acc = np.equal(t,test_y).mean() * 100
    text = "The Linear Regression's accuracy is %.2f%%." %(acc)
    print(text)
    er = t - y
    MSE =  ( er**2 ).mean()
    MAE = abs( er ).mean()
    SD = np.sqrt((( er-er.mean())**2).mean())
    obj = pd.Series( [MSE,MAE,SD],index=['MSE','MAE','SD'] )
    print(obj)


In [4]:
data_path = "E://yuwoliang/three set/SpectralClassification/No removal of continuous spectrum/"

data=sio.loadmat( data_path + "SpectralClassificationTrain.mat" ) 
train_x = []
train_y = data['train_y'][:,0]
flag = np.unique(train_y)
for index in flag:
    train_x.append(data['train_x'][train_y==index,:])

data=sio.loadmat( data_path + "SpectralClassificationTest.mat" ) 
test_x = data['test_x']
test_y = data['test_y'][:,0]

del data

In [5]:
[w , flag ] = LDA_tr( train_x )
LDA_te( test_x , test_y , w , flag )

The Linear Regression's accuracy is 81.00%.
MSE    0.205000
MAE    0.195000
SD     0.446514
dtype: float64
